In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import re
import nltk
import math
import pickle

import xgboost

from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.linear_model import Perceptron

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from Bio.SeqUtils.ProtParam import ProteinAnalysis
#from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, cross_val_predict

%matplotlib inline

In [2]:
# read in data from csv file
df = pd.read_csv('proteins-5-functions.csv')

df.head()

,classification,name,sequence,structureId,type
0,TRANSFERASE,NaN,MEIYEGKLTAEGLRFGIVASRFNHALVDRLVEGAIDCIVRHGGREE...,5MPP,4
1,HYDROLASE,NaN,MKFTLTIAGLLAVGSTAAPTTEKRNPGGIDYVQNYNGDVADFQYNE...,3M4F,0
2,TRANSFERASE,NaN,MRGSHHHHHHGSMKRAVITGLGIVSSIGNNQQEVLASLREGRSGIT...,2BYY,4
3,HYDROLASE,NaN,STGSATTTPIDSLDDAYITPVQIGTPAQTLNLDFDTGSSDLWVFSS...,4YCY,0
4,TRANSFERASE,NaN,GSGMMRYLHKIELELNRLTSRYPFFKKIAFDAEIIKLVDDLNVDEN...,3AQC,4


In [3]:
# read in more immune system sequences data from csv file
df_2 = pd.read_csv('uniprot-human-immune.csv')

df_2.head()

,name,sequence,classification,type
0,tr|Q6FGW4|Q6FGW4_HUMAN,MHSSALLCCLVLLTGVRASPGQGTQSENSCTHFPGNLPNMLRDLRD...,IMMUNE SYSTEM,1
1,tr|A0A024R5Z3|A0A024R5Z3_HUMAN,MSSILPFTPPIVKRLLGWKKGEQNGQEEKWCEKAVKSLVKKLKKTG...,IMMUNE SYSTEM,1
2,tr|A0A0B4Q6D0|A0A0B4Q6D0_9PLVG,MRVKGIRKNYQHLWRGGTLLLGMLMICSAVEKLWVTVYYGVPVWKE...,IMMUNE SYSTEM,1
3,tr|A0A0B4Q7W6|A0A0B4Q7W6_9PLVG,MRVKGIRKNYQHLWRGGTLLLGMLMICSAVEKLWVTVYYGVPVWKE...,IMMUNE SYSTEM,1
4,tr|A0A0B4Q6H6|A0A0B4Q6H6_9PLVG,MRVKGIRKNYQHLWRGGTLLLGMLMICSAVEKLWVTVHYGVPVWKE...,IMMUNE SYSTEM,1


In [4]:
df_2.shape

(84798, 4)

In [5]:
df3 = df_2.sample(frac=0.33)
df3.shape

(27983, 4)

In [6]:
df_hydro = pd.read_csv('uniprot-hydrolase-reviewed.csv')

df_hydro.head()

,name,sequence,classification,type
0,sp|P9WNH5|HSAD_MYCTU,MTATEELTFESTSRFAEVDVDGPLKLHYHEAGVGNDQTVVLLHGGG...,HYDROLASE,0
1,sp|P77044|MHPC_ECOLI,MSYQPQTEAATSRFLNVEEAGKTLRIHFNDCGQGDETVVLLHGSGP...,HYDROLASE,0
2,sp|Q75UV1|NDX1_THETH,MELGAGGVVFNAKREVLLLRDRMGFWVFPKGHPEPGESLEEAAVRE...,HYDROLASE,0
3,sp|Q60928|GGT1_MOUSE,MKNRFLVLGLVAVVLVFVIIGLCIWLPYTSGKPDHVYSRAAVATDA...,HYDROLASE,0
4,sp|P19440|GGT1_HUMAN,MKKKLVVLGLLAVVLVLVIVGLCLWLPSASKEPDNHVYTRAAVAAD...,HYDROLASE,0


In [7]:
df_h2 = df_hydro.sample(frac=0.45)
df_h2.shape

(26386, 4)

In [8]:
df_ox = pd.read_csv('uniprot-oxidoreductase-Feb1.csv')

df_ox.head()

,name,sequence,classification,type
0,sp|Q8L540|LTO1_ARATH,MMARFVSVSSCQFHFGFREVSPPSVTSYPRRFEVSDRRFPAIPIKC...,OXIDOREDUCTASE,5
1,sp|Q9AL95|NROR_CLOAB,MKSTKILILGAGPAGFSAAKAALGKCDDITMINSEKYLPYYRPRLN...,OXIDOREDUCTASE,5
2,sp|P55931|ETFD_PIG,MMVPLAKLASPAYQCFHALKIKKNYLPLCATRWSSTCKVPRITTHY...,OXIDOREDUCTASE,5
3,sp|Q16134|ETFD_HUMAN,MLVPLAKLSCLAYQCFHALKIKKNYLPLCATRWSSTSTVPRITTHY...,OXIDOREDUCTASE,5
4,sp|Q9NZC7|WWOX_HUMAN,MAALRYAGLDDTDSEDELPPGWEERTTKDGWVYYANHTEEKTQWEH...,OXIDOREDUCTASE,5


In [9]:
df_FeS = pd.read_csv('uniprot-ironSulfurCluster-Feb1.csv')

df_FeS.head()

,name,sequence,classification,type
0,tr|T0M7Z9|T0M7Z9_CAMFR,MAAVVAFSLRRRFPATTLGGACLQQPWLALQDSLARCVKPTGRDCK...,IRON-SULFUR-CLUSTER,6
1,tr|A0A1Q9C7M1|A0A1Q9C7M1_SYMMI,MSLAASLRSLAPRGLGRSGLRGFAAMAQPQTFAIYRYDPDQQAKPF...,IRON-SULFUR-CLUSTER,6
2,tr|A0A2P6TTG2|A0A2P6TTG2_CHLSO,MLSLIRQAGSLGQALCPELTQLVRTVASAAGAAGGASKPVVEKEFL...,IRON-SULFUR-CLUSTER,6
3,tr|A0A091CKG8|A0A091CKG8_FUKDA,MAAVAGFSLRRRFPATVLGGSCLQACRGAQTAADRAPRIKKFAIYR...,IRON-SULFUR-CLUSTER,6
4,tr|Q59QN7|Q59QN7_CANAL,MFRSILHQQKAVQFSVRSLATAAAEKAPRLKKFQIYRWNPDTPEVQ...,IRON-SULFUR-CLUSTER,6


In [10]:
df_gpcr = pd.read_csv('uniprot-gpcr-unreviewed.csv')

df_gpcr.head()

,name,sequence,classification,type
0,tr|G3V654|G3V654_RAT,MAATTPAQDVGVEIYLGPVWPAPSNSTPLALNLSLALREDAPGNLT...,GPCR,8
1,tr|A0A287DFW8|A0A287DFW8_ICTTR,MDVTSPAHTFGMEIYPRTTQLADSNSTSPELNLSLPLQGTALVNQT...,GPCR,8
2,tr|A0A0R4J0N8|A0A0R4J0N8_MOUSE,MEPISVSIYTSDNYSEEVGSGDYDSNKEPCFRDENVHFNRIFLPTI...,GPCR,8
3,tr|A0A1U7QAF5|A0A1U7QAF5_MESAU,MDPLNLSWYDDDLERQNWSRPFNGSEGKADRPHYNYYAMLLTLLIF...,GPCR,8
4,tr|A0A0N8ETT2|A0A0N8ETT2_HETGA,MDPLNLTWYDDDLERQNWSRSFNGSEGKADRPHYNYYAMLLTLLIF...,GPCR,8


In [11]:
df_gp2 = df_gpcr.sample(frac=0.16)

df_gp2.shape

(31401, 4)

In [12]:
df_lb = pd.read_csv('uniprot-lipidBinding-Feb5.csv')
df_lb2 = df_lb.sample(frac=0.5)

df_lb2.shape

(31304, 4)

In [13]:
df.columns

Index([u'classification', u'name', u'sequence', u'structureId', u'type'], dtype='object')

In [14]:
df.shape

(104956, 5)

In [15]:
# concat the dataframes
df_ = pd.concat([df,df3,df_h2,df_ox,df_FeS,df_gp2,df_lb2])

df_.head()

/Users/scottlew/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,classification,name,sequence,structureId,type
0,TRANSFERASE,NaN,MEIYEGKLTAEGLRFGIVASRFNHALVDRLVEGAIDCIVRHGGREE...,5MPP,4
1,HYDROLASE,NaN,MKFTLTIAGLLAVGSTAAPTTEKRNPGGIDYVQNYNGDVADFQYNE...,3M4F,0
2,TRANSFERASE,NaN,MRGSHHHHHHGSMKRAVITGLGIVSSIGNNQQEVLASLREGRSGIT...,2BYY,4
3,HYDROLASE,NaN,STGSATTTPIDSLDDAYITPVQIGTPAQTLNLDFDTGSSDLWVFSS...,4YCY,0
4,TRANSFERASE,NaN,GSGMMRYLHKIELELNRLTSRYPFFKKIAFDAEIIKLVDDLNVDEN...,3AQC,4


In [16]:
df_.shape

(293026, 5)

## Remove duplicate protein sequences from data

In [17]:
# remove the duplicate protein sequences
df_ = df_.drop_duplicates(subset='sequence', keep="first")

df_.shape

(289029, 5)

## 5 Classes of Proteins

In [18]:
df_.classification.value_counts()

OXIDOREDUCTASE         37569
HYDROLASE              37410
TRANSFERASE            35880
LIGASE                 33967
IMMUNE SYSTEM          33437
GPCR                   31400
LIPID BINDING          31193
IRON-SULFUR-CLUSTER    31185
ISOMERASE              16988
Name: classification, dtype: int64

In [19]:
df_.type.value_counts()

5    37569
0    37410
4    35880
3    33967
1    33437
8    31400
9    31193
6    31185
2    16988
Name: type, dtype: int64

## Subset the data: use fewer rows of data for quicker comparison of XGBoost with other models

In [20]:
df_new= df_.sample(frac = 1.0)

df_new.shape

(289029, 5)

## Classifier of Protein Sequences: Predict 9 Classes

## CountVectorizer: Count peptide frequency, transform the data

In [21]:
# In this case, peptide frequency is used for analysis
peptide_size = 5
vect_ = CountVectorizer(min_df=1,token_pattern=r'\w{1}',ngram_range=(peptide_size,peptide_size))

## Split the data into training & test sets for classification model

In [22]:
X = vect_.fit_transform(df_new.sequence)
y = df_new.type

In [23]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state =42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

((231223, 2991414), (231223,))
((57806, 2991414), (57806,))


In [24]:
y_test.value_counts()

5    7542
0    7478
4    7188
3    6753
1    6580
9    6407
8    6245
6    6141
2    3472
Name: type, dtype: int64

## Classification Models

## MNB Classifier Model

In [25]:
%%time
mnb = MultinomialNB(alpha=50)
mnb.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("MNB classifier Score: {:.2f}".format(mnb.score(X_test, y_test)))
print()

MNB classifier Score: 0.87
()
CPU times: user 8.78 s, sys: 1.74 s, total: 10.5 s
Wall time: 11 s


In [26]:
%%time
pac = PassiveAggressiveClassifier()
pac.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Passive Aggressive classifier Score: {:.2f}".format(pac.score(X_test, y_test)))
print()

/Users/scottlew/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in PassiveAggressiveClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Passive Aggressive classifier Score: 0.94
()
CPU times: user 1min 7s, sys: 1.78 s, total: 1min 8s
Wall time: 1min 12s


In [27]:
%%time
pac2 = PassiveAggressiveClassifier(loss='squared_hinge')
pac2.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Passive Aggressive classifier with squared hinge loss Score: {:.2f}".format(pac2.score(X_test, y_test)))
print()

Passive Aggressive classifier with squared hinge loss Score: 0.93
()
CPU times: user 1min 1s, sys: 1.13 s, total: 1min 2s
Wall time: 1min 3s


In [28]:
%%time
sgd = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("SGD classifier Score: {:.2f}".format(sgd.score(X_test, y_test)))
print()

SGD classifier Score: 0.93
()
CPU times: user 1min 54s, sys: 1.45 s, total: 1min 56s
Wall time: 1min 57s


In [29]:
%%time
sgd2 = linear_model.SGDClassifier(loss= 'log',max_iter=1000, tol=1e-3)
sgd2.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("SGD classifier Score: {:.2f}".format(sgd2.score(X_test, y_test)))
print()

SGD classifier Score: 0.94
()
CPU times: user 2min 57s, sys: 2.63 s, total: 2min 59s
Wall time: 3min 6s


In [30]:
%%time
per = Perceptron(tol=1e-3, random_state=0).fit(X_train,y_train)
# evaluate accuracy of our model on test data
print("Perceptron classifier Score: {:.2f}".format(per.score(X_test, y_test)))
print()

Perceptron classifier Score: 0.93
()
CPU times: user 2min 54s, sys: 3.38 s, total: 2min 58s
Wall time: 3min 14s


In [31]:
# Generate Confusion Matrix for Perceptron Model
actual = y_test
predictions = per.predict(X_test)
print('Confusion Matrix for Perceptron Model')
print()
cm = confusion_matrix(actual,predictions)
print(cm)

Confusion Matrix for Perceptron Model
()
[[6409   88   89  138  359  169  129   44   53]
 [  73 6404    7   24   35   13   13    5    6]
 [  59   10 3218   30   77   38   30    5    5]
 [ 147   11   31 6255  150   37   66   18   38]
 [ 391   57   64  130 6114  149  147   54   82]
 [ 195   29   50   64  164 6872   92   40   36]
 [   6    0    0    1    3    5 6125    0    1]
 [  10    0    2   12    6   10    3 6196    6]
 [  49    1    7   15   39   18   20    9 6249]]


## Voting Models

In [32]:
%%time
clf1 = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2 = PassiveAggressiveClassifier(C=0.1)

eclf1 = VotingClassifier(estimators=[('sgd',clf1), ('pac',clf2)], voting='hard')
eclf1.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf1.score(X_test, y_test)))
print()

Majority Voting Classifier Score: 0.93
()
CPU times: user 2min 52s, sys: 4.71 s, total: 2min 57s
Wall time: 2min 53s


In [33]:
# Generate Confusion Matrix for Voting Model
actual = y_test
predictions = eclf1.predict(X_test)
print('Confusion Matrix for Voting Model')
print()
cm = confusion_matrix(actual,predictions)
print(cm)

Confusion Matrix for Voting Model
()
[[6984   23    8  100  266   88    0    1    8]
 [ 252 6303    0   10    8    6    0    1    0]
 [ 227    3 3121   21   67   32    0    1    0]
 [ 436    7    4 6134  132   29    0    4    7]
 [ 699   29   13  114 6219   84    0    0   30]
 [ 429    1    5   38  189 6871    7    1    1]
 [  25    0    0    0    5    6 6105    0    0]
 [ 105    3    0   21   28   33    0 6055    0]
 [ 178    0    1   19   46   30    0    2 6131]]


In [34]:
%%time
clf1a = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2a = PassiveAggressiveClassifier(C=0.1)
clf3a = MultinomialNB(alpha=50)

eclf2 = VotingClassifier(estimators=[('sgd', clf1a), ('pac', clf2a), ('mnb', clf3a)], voting='hard')
eclf2.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf2.score(X_test, y_test)))
print()

Majority Voting Classifier Score: 0.94
()
CPU times: user 2min 59s, sys: 3.85 s, total: 3min 3s
Wall time: 2min 59s


In [35]:
# Generate Confusion Matrix for Voting Model
actual = y_test
predictions = eclf2.predict(X_test)
print('Confusion Matrix for Voting Model')
print()
cm = confusion_matrix(actual,predictions)
print(cm)

Confusion Matrix for Voting Model
()
[[6841   22    6  160  338   95    0    4   12]
 [ 215 6327    1   18   13    4    0    2    0]
 [ 212    2 3099   32   94   32    0    1    0]
 [ 272    5    1 6288  151   24    0    5    7]
 [ 586   29   11  145 6291   86    0    3   37]
 [ 405    1    4   80  155 6881    7    3    6]
 [  19    0    0    1    4    3 6114    0    0]
 [  20    1    0   12    8    6    0 6198    0]
 [ 152    0    0   38   38   11    0    2 6166]]


In [36]:
%%time
clf1b = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2b = PassiveAggressiveClassifier(C=0.1)
clf3b = linear_model.SGDClassifier(loss= 'log',max_iter=1000, tol=1e-3)

eclf3 = VotingClassifier(estimators=[('sgd', clf1b), ('pac', clf2b), ('sgd_lg', clf3b)], voting='hard')
eclf3.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf3.score(X_test, y_test)))
print()

Majority Voting Classifier Score: 0.94
()
CPU times: user 5min 1s, sys: 5.21 s, total: 5min 6s
Wall time: 5min


In [37]:
%%time
clf1c = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2c = PassiveAggressiveClassifier(C=0.1)
clf3c = Perceptron(tol=1e-3, random_state=0)

eclf4 = VotingClassifier(estimators=[('sgd', clf1c), ('pac', clf2c), ('per', clf3c)], voting='hard')
eclf4.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf4.score(X_test, y_test)))
print()

Majority Voting Classifier Score: 0.94
()
CPU times: user 5min 20s, sys: 4.64 s, total: 5min 25s
Wall time: 5min 19s


In [38]:
vote_prediction4 = eclf4.predict(X_test)
print(classification_report(vote_prediction4, y_test))

              precision    recall  f1-score   support

           0       0.91      0.82      0.86      8351
           1       0.97      0.98      0.98      6518
           2       0.92      0.98      0.95      3248
           3       0.93      0.96      0.94      6511
           4       0.89      0.88      0.88      7256
           5       0.92      0.96      0.94      7304
           6       1.00      1.00      1.00      6136
           8       0.99      1.00      0.99      6201
           9       0.97      0.99      0.98      6281

   micro avg       0.94      0.94      0.94     57806
   macro avg       0.94      0.95      0.95     57806
weighted avg       0.94      0.94      0.94     57806



In [39]:
%%time
clf1d = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2d = PassiveAggressiveClassifier(C=0.1)
clf3d = Perceptron(tol=1e-3, random_state=0)
clf4d = MultinomialNB(alpha=50)

eclf5 = VotingClassifier(estimators=[('sgd', clf1d), ('pac', clf2d), ('per', clf3d),('mnb',clf4d)], voting='hard')
eclf5.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf5.score(X_test, y_test)))
print()

Majority Voting Classifier Score: 0.94
()
CPU times: user 5min 51s, sys: 8.56 s, total: 6min
Wall time: 6min 7s


In [40]:
vote_prediction5 = eclf5.predict(X_test)
print(classification_report(vote_prediction5, y_test))

              precision    recall  f1-score   support

           0       0.92      0.81      0.86      8446
           1       0.97      0.98      0.98      6484
           2       0.91      0.99      0.95      3201
           3       0.94      0.93      0.94      6809
           4       0.87      0.90      0.88      6973
           5       0.92      0.96      0.94      7248
           6       1.00      1.00      1.00      6129
           8       0.99      0.99      0.99      6241
           9       0.97      0.99      0.98      6275

   micro avg       0.94      0.94      0.94     57806
   macro avg       0.94      0.95      0.95     57806
weighted avg       0.94      0.94      0.94     57806



## Cross Validation

In [41]:
clf1e = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2e = PassiveAggressiveClassifier(C=0.1)

eclf_cv = VotingClassifier(estimators=[('sgd',clf1e), ('pac',clf2e)], voting='hard')
scores = cross_val_score(eclf_cv,X,y, cv = 5)
print("Cross-validation scores for Voting Classifier: {}".format(scores))
print()
print("The average accuracy score for Voting Classifier is: ")
print(np.mean(scores))

Cross-validation scores for Voting Classifier: [0.93305425 0.93256989 0.93343252 0.93277282 0.93152605]
()
The average accuracy score for Voting Classifier is: 
0.9326711029149324


In [42]:
clf1f = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2f = PassiveAggressiveClassifier(C=0.1)
clf3f = Perceptron(tol=1e-3, random_state=0)

eclf_cv2 = VotingClassifier(estimators=[('sgd',clf1f), ('pac',clf2f),('per', clf3f)], voting='hard')
scores = cross_val_score(eclf_cv2,X,y, cv = 5)

print("Cross-validation scores for Voting Classifier: {}".format(scores))
print()
print("The average accuracy score for Voting Classifier is: ")
print(np.mean(scores))

Cross-validation scores for Voting Classifier: [0.94326045 0.94355453 0.94485002 0.94190713 0.94071242]
()
The average accuracy score for Voting Classifier is: 
0.9428569087000686


In [43]:
clf1g = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2g = PassiveAggressiveClassifier(C=0.1)
clf3g = Perceptron(tol=1e-3, random_state=0)
clf4g = MultinomialNB(alpha=50)

eclf_cv3 = VotingClassifier(estimators=[('sgd',clf1g), ('pac',clf2g),('per', clf3g),('mnb',clf4g)], voting='hard')
scores3 = cross_val_score(eclf_cv3,X,y, cv = 5)

print("Cross-validation scores for Voting Classifier: {}".format(scores3))
print()
print("The average accuracy score for Voting Classifier is: ")
print(np.mean(scores3))

Cross-validation scores for Voting Classifier: [0.94282798 0.94291448 0.94376016 0.94185523 0.94031452]
()
The average accuracy score for Voting Classifier is: 
0.9423344745151159


In [44]:
#######################################################################################